<a href="https://colab.research.google.com/github/DarkWindman/DataAnalisys_Labs_2023/blob/main/lab3_intlel_analisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [165]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
df = pd.read_csv('/content/sample_data/data.csv')
df.shape

(748, 5)

In [166]:
fname = "/content/sample_data/data.csv"
all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),"whether he/she donated blood in March 2007"
EXAMPLE FEATURES: [2.0, 50.0, 12500.0, 98.0]
features.shape: (748, 4)
targets.shape: (748, 1)


In [167]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 599
Number of validation samples: 149


In [168]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 164 (27.38% of total)


In [169]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [170]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 256)               1280      
                                                                 
 dense_41 (Dense)            (None, 256)               65792     
                                                                 
 dropout_25 (Dropout)        (None, 256)               0         
                                                                 
 dense_42 (Dense)            (None, 256)               65792     
                                                                 
 dropout_26 (Dropout)        (None, 256)               0         
                                                                 
 dense_43 (Dense)            (None, 1)                 257       
                                                                 
Total params: 133121 (520.00 KB)
Trainable params: 13

In [183]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"]
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=15,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/15
1/1 - 2s - loss: 0.0018 - accuracy: 0.7329 - val_loss: 0.4179 - val_accuracy: 0.8993 - 2s/epoch - 2s/step
Epoch 2/15
1/1 - 0s - loss: 0.0018 - accuracy: 0.7195 - val_loss: 0.4178 - val_accuracy: 0.8993 - 72ms/epoch - 72ms/step
Epoch 3/15
1/1 - 0s - loss: 0.0018 - accuracy: 0.7195 - val_loss: 0.4177 - val_accuracy: 0.8993 - 81ms/epoch - 81ms/step
Epoch 4/15
1/1 - 0s - loss: 0.0018 - accuracy: 0.7295 - val_loss: 0.4178 - val_accuracy: 0.8993 - 81ms/epoch - 81ms/step
Epoch 5/15
1/1 - 0s - loss: 0.0018 - accuracy: 0.7295 - val_loss: 0.4177 - val_accuracy: 0.8993 - 88ms/epoch - 88ms/step
Epoch 6/15
1/1 - 0s - loss: 0.0018 - accuracy: 0.7362 - val_loss: 0.4177 - val_accuracy: 0.8993 - 75ms/epoch - 75ms/step
Epoch 7/15
1/1 - 0s - loss: 0.0018 - accuracy: 0.7295 - val_loss: 0.4177 - val_accuracy: 0.8993 - 90ms/epoch - 90ms/step
Epoch 8/15
1/1 - 0s - loss: 0.0018 - accuracy: 0.7245 - val_loss: 0.4177 - val_accuracy: 0.8993 - 76ms/epoch - 76ms/step
Epoch 9/15
1/1 - 0s - loss: 0.0018 -

## **test photo**

In [108]:
import tensorflow as tf
from tensorflow.keras import layers
import zipfile
import os
zip_ref = zipfile.ZipFile('/content/sample_data/Dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content/sample_data') #Extracts the files into the /tmp folder
zip_ref.close()
import os

num_skipped = 0
for folder_name in ("Bus", "Car", "Truck", "motorcycle"):
    folder_path = os.path.join("/content/sample_data/Dataset", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 61 images


In [109]:
image_size = (180, 180)
batch_size = 128
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/sample_data/Dataset",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 339 files belonging to 4 classes.
Using 272 files for training.
Using 67 files for validation.


In [110]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal", input_shape=(180, 180, 3)),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
  ])

In [111]:
AUTOTUNE = tf.data.AUTOTUNE

train = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [147]:
model_2 = keras.Sequential([
  data_augmentation,
  keras.layers.Rescaling(1./255),
  keras.layers.Conv2D(64, 3, strides=2, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Conv2D(128, 3, strides=2, padding='same', activation='tanh'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Dropout(0.3),
  keras.layers.Conv2D(256, 3, strides=2, padding='same', activation='tanh'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
  keras.layers.BatchNormalization(),
  keras.layers.Flatten(),
  keras.layers.Dense(256, activation='softmax'),
  keras.layers.Dense(4, name="outputs")
])


In [148]:
model_2.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x7ba3bf25c760>>

In [152]:
model_2.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0002), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])


model_2.fit(train, validation_data = val_ds, epochs=10)

Epoch 1/10
3/3 [==============================] - 12s 3s/step - loss: 1.2734 - accuracy: 0.6985 - val_loss: 1.3841 - val_accuracy: 0.2388
Epoch 2/10
3/3 [==============================] - 7s 3s/step - loss: 1.2747 - accuracy: 0.6507 - val_loss: 1.3838 - val_accuracy: 0.2537
Epoch 3/10
3/3 [==============================] - 7s 3s/step - loss: 1.2762 - accuracy: 0.6691 - val_loss: 1.3836 - val_accuracy: 0.2388
Epoch 4/10
3/3 [==============================] - 9s 3s/step - loss: 1.2661 - accuracy: 0.7279 - val_loss: 1.3831 - val_accuracy: 0.2537
Epoch 5/10
3/3 [==============================] - 7s 3s/step - loss: 1.2688 - accuracy: 0.6838 - val_loss: 1.3826 - val_accuracy: 0.2537
Epoch 6/10
3/3 [==============================] - 9s 2s/step - loss: 1.2658 - accuracy: 0.7096 - val_loss: 1.3824 - val_accuracy: 0.2537
Epoch 7/10
3/3 [==============================] - 7s 3s/step - loss: 1.2660 - accuracy: 0.7169 - val_loss: 1.3820 - val_accuracy: 0.2537
Epoch 8/10
3/3 [========================

## Ex/3 Test

In [115]:
df1 = pd.read_csv('/content/sample_data/ireland-news-headlines.csv')
df1.columns

Index(['publish_date', 'headline_category', 'headline_text'], dtype='object')

In [116]:
df2 = df1[0:5000]

In [117]:
x_train, x_test, y_train, y_test = train_test_split(df2['headline_text'], df2['headline_category'], test_size=0.20)


In [118]:
from collections import Counter
Counter( df2['headline_category'])

Counter({'news': 2502,
         'culture': 321,
         'opinion': 201,
         'opinion.letters': 360,
         'business': 647,
         'sport': 807,
         'news.health': 15,
         'news.education': 81,
         'lifestyle.homes': 32,
         'news.politics.oireachtas': 34})

In [119]:
for i in range(len(y_train)):
  y_train[y_train=='news']=1
  y_train[y_train=='business']=2
  y_train[y_train=='sport']=3
  y_train[y_train=='opinion.letters']=4
  y_train[y_train=='news.politics.oireachtas']=1
  y_train[y_train=='lifestyle.homes']=4
  y_train[y_train=='news.education']=1
  y_train[y_train=='news.health']=2
  y_train[y_train=='opinion']=4
  y_train[y_train=='culture']=4



In [120]:
for i in range(len(y_train)):
  y_test[y_test=='news']=1
  y_test[y_test=='business']=2
  y_test[y_test=='sport']=3
  y_test[y_test=='opinion.letters']=4
  y_test[y_test=='news.politics.oireachtas']=1
  y_test[y_test=='lifestyle.homes']=4
  y_test[y_test=='news.education']=1
  y_test[y_test=='news.health']=2
  y_test[y_test=='opinion']=4
  y_test[y_test=='culture']=4

In [121]:
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [122]:
print(y_train)

[2. 2. 2. ... 1. 1. 1.]


In [123]:
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 5000
oov_token = ""
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x_train)

In [124]:
x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

In [125]:
max_length = 100
padding_type = "post"
trunction_type="post"
x_train_padded = keras.utils.pad_sequences(x_train_sequences,maxlen=max_length, padding=padding_type,
                       truncating=trunction_type)
x_test_padded = keras.utils.pad_sequences(x_test_sequences,maxlen=max_length,
                               padding=padding_type, truncating=trunction_type)

In [129]:
word_index = tokenizer.word_index
total_words = len(tokenizer.word_index) + 1

In [130]:
EMBED_DIM = 32
LSTM_OUT = 64
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='selu'))

In [131]:
model.compile(optimizer='adam',
                loss = tf.keras.losses.binary_crossentropy,
                metrics=['accuracy'])

In [132]:
model.fit(x_train_padded, y_train, epochs=20, validation_data=(x_test_padded, y_test))

Epoch 1/20
125/125 [==============================] - 11s 73ms/step - loss: -13.4975 - accuracy: 0.4840 - val_loss: -15.7830 - val_accuracy: 0.5120
Epoch 2/20
125/125 [==============================] - 7s 60ms/step - loss: -15.1844 - accuracy: 0.5263 - val_loss: -15.7830 - val_accuracy: 0.5120
Epoch 3/20
125/125 [==============================] - 7s 59ms/step - loss: -15.1844 - accuracy: 0.5263 - val_loss: -15.7830 - val_accuracy: 0.5120
Epoch 4/20
125/125 [==============================] - 8s 63ms/step - loss: -15.1844 - accuracy: 0.5263 - val_loss: -15.7830 - val_accuracy: 0.5120
Epoch 5/20
125/125 [==============================] - 8s 62ms/step - loss: -15.1844 - accuracy: 0.5263 - val_loss: -15.7830 - val_accuracy: 0.5120
Epoch 6/20
125/125 [==============================] - 8s 68ms/step - loss: -15.1844 - accuracy: 0.5263 - val_loss: -15.7830 - val_accuracy: 0.5120
Epoch 7/20
125/125 [==============================] - 7s 56ms/step - loss: -15.1844 - accuracy: 0.5263 - val_loss: -1